In [1]:
import pickle

with open('classification_dat.pkl', 'rb') as f:  # open a text file
    classification_df = pickle.load(f)

with open('regression_dat.pkl', 'rb') as f:  # open a text file
    regression_df = pickle.load(f)

In [2]:
from hana_ml import dataframe

config = 'hana-build.hana-ml.c.ap-cn-1.cloud.sap', 31315, 'xxxxx', 'xxxxx'
connection_context = dataframe.ConnectionContext(address=config[0], port=config[1], user=config[2], password=config[3])

In [ ]:
from timeit import time
import json
from hana_ml.algorithms.pal.auto_ml import AutomaticClassification
from hana_ml.visualizers.automl_progress import PipelineProgressStatusMonitor

for case in classification_df:
    dataframe.create_dataframe_from_pandas(connection_context, case["pandas"], table_name=case["file_name"], force=True)
    hana_df = connection_context.table(case["file_name"])
    start_time = time.time()
    auto_c = AutomaticClassification(connections='default')
    auto_c.disable_workload_class_check()
    progress_status_monitor = PipelineProgressStatusMonitor(connection_context=connection_context,
                                                            automatic_obj=auto_c,
                                                            runtime_platform='console')
    progress_status_monitor.start()
    auto_c.fit(hana_df, key=case["key"], label=case["label"])
    result = {
        "file_name": case["file_name"],
        "metrics": json.loads(auto_c.best_pipeline_.SCORES.collect().iat[0,0]),
        "runtime": time.time() - start_time
    }
    print(result)
    with open('hana_{}_result.pkl'.format(case["file_name"]), 'wb') as f:  # open a text file
        pickle.dump(result, f)

Progress Indicator Id:  AutoML-7CF4CC7D_B8D2_11EE_B545_F47B099F40D8
Creation Time:  2024-01-22 03:01:15.600000
Function Name:  AUTOML_FIT
+---------+-----------------+-------------------+--------------------+----------+
|  Status | Pipeline Number | Generation Number | Current Generation | Progress |
+---------+-----------------+-------------------+--------------------+----------+
| Running |        62       |         5         |   Initialization   |   0 %    |
+---------+-----------------+-------------------+--------------------+----------+

Current Generation Details:  Initialization
+------------+-----------+-----------+--------+---------+
| Evaluating | Evaluated | Succeeded | Failed | Timeout |
+------------+-----------+-----------+--------+---------+
|     20     |     17    |     15    |   2    |    0    |
+------------+-----------+-----------+--------+---------+
Evaluated(succeeded) {'DT_Classifier': {'args': {'ALGORITHM': 1, 'MAX_DEPTH': 3, 'MIN_RECORDS_OF_PARENT': 7, 'MIN_REC

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\I308290\AppData\Local\miniconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\users\i308290\hanamlapi\src\hana_ml\visualizers\automl_progress.py", line 546, in run
    self.fetch()
  File "c:\users\i308290\hanamlapi\src\hana_ml\visualizers\automl_progress.py", line 503, in fetch
ERROR:hana_ml.algorithms.pal.auto_ml:(-10807, "Connection down: [89006] System call 'recv' failed, rc=10054:远程主机强迫关闭了一个现有的连接。 {0.0.0.0:60056 -> 10.237.43.114:31315 TenantName:(none) SiteVolumeID:0:2 SiteType:NONE ConnectionID:208429 SessionID:1034884207529778}")
Traceback (most recent call last):
  File "c:\users\i308290\hanamlapi\src\hana_ml\algorithms\pal\auto_ml.py", line 781, in _fit
    self._call_pal_auto(conn,
  File "c:\users\i308290\hanamlapi\src\hana_ml\algorithms\pal\pal_base.py", line 892, in _call_pal_auto
    self.execute_statement, materialize_dict = call_pal_auto_with_hint(conn_contex

In [ ]:
from timeit import time
import json
from hana_ml.algorithms.pal.auto_ml import AutomaticRegression
from hana_ml.visualizers.automl_progress import PipelineProgressStatusMonitor
for case in regression_df:
    if not connection_context.has_table(case["file_name"]):
        dataframe.create_dataframe_from_pandas(connection_context, case["pandas"], table_name=case["file_name"])
    hana_df = connection_context.table(case["file_name"])
    start_time = time.time()
    auto_r = AutomaticRegression(connections='default')
    auto_r.disable_workload_class_check()
    progress_status_monitor = PipelineProgressStatusMonitor(connection_context=connection_context,
                                                            automatic_obj=auto_r,
                                                            runtime_platform='console')
    progress_status_monitor.start()
    auto_r.fit(hana_df, key=case["key"], label=case["label"])
    result = {
        "file_name": case["file_name"],
        "metrics": json.loads(auto_r.best_pipeline_.SCORES.collect().iat[0,0]),
        "runtime": time.time() - start_time
    }
    print(result)
    with open('hana_{}_result.pkl'.format(case["file_name"]), 'wb') as f:  # open a text file
        pickle.dump(result, f)